In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
## Reading the merged, pivoted datasets
data_merged_MP = pd.read_csv('data_merged_MP.csv', index_col=0)
data_merged_NoS = pd.read_csv('data_merged_NoS.csv', index_col=0)
data_merged_Ratio = pd.read_csv('data_merged_Ratio.csv', index_col=0)

In [15]:
## Labels

fraud_database = pd.read_csv('../Labels/CA_Ophthalmology/Fraud_Database.csv', index_col=0)
high_utilization_database = pd.read_csv('../Labels/CA_Ophthalmology/HighUtilization_Database.csv', index_col=0)
non_fraud_database = pd.read_csv('../Labels/CA_Ophthalmology/NonFraud_Database.csv', index_col=0)
mislabeled_database = pd.read_csv('../Labels/CA_Ophthalmology/Mislabeled_Database.csv', index_col=0)

fraud = list(fraud_database.index)
high_utilization = list(high_utilization_database.index)
non_fraud = list(non_fraud_database.index)
mislabeled = list(mislabeled_database.index)

data_merged = [data_merged_MP.copy(), data_merged_NoS.copy(), data_merged_Ratio.copy()]

## Dropping the mislabeled providers from the datasets
for i in range(len(data_merged)):
    data_merged[i] = data_merged[i].drop(index=mislabeled)

flag = pd.Series(data=['unlabeled'] * len(data_merged[0].index), index=data_merged[0].index, dtype='str', name='flag')
for npi in non_fraud:
     flag.at[npi] = 'non-fraudulent'
for npi in high_utilization:
    flag.at[npi] = 'high utilization'
for npi in fraud:
    flag.at[npi] = 'fraudulent'
    
[x_MP, x_NoS, x_Ratio] = data_merged
x_MP_train, x_MP_unlabeled = x_MP.loc[flag != 'unlabeled'], x_MP.loc[flag == 'unlabeled']
x_NoS_train, x_NoS_unlabeled = x_NoS.loc[flag != 'unlabeled'], x_NoS.loc[flag == 'unlabeled']
x_Ratio_train, x_Ratio_unlabeled = x_Ratio.loc[flag != 'unlabeled'], x_Ratio.loc[flag == 'unlabeled']

y_train = flag[flag != 'unlabeled']

In [101]:
from scipy.stats import ttest_ind
from collections import defaultdict

significant = defaultdict(int)

for dataset in data_merged:
    pvalues = {}
    for f in dataset.columns:
        group1 = dataset.loc[flag == 'fraudulent', f]
        group2 = dataset.loc[flag == 'non-fraudulent', f]
        val = ttest_ind(group1.values, group2.values)[1]
        if str(val) != 'nan' :
            pvalues[f] = val
            
    p = 0.1/len(dataset.columns)

    for f in pvalues:
        if pvalues[f] < p:
            significant[f] += 1

In [106]:
used_codes = ['14061', '65800', '65855', '67973', '67975', '68326', '68335', '76510', '76511', '76512', 
              '76513', '92060', '92100', '92235', '92240', '92275', '92284', '95004', '95930', '65100', 
              '65210', '65222', '65400', '65426', '65722', '65815', '66030', '66175', '66183', '66761', 
              '66982', '67938', '68761', '68801', '76514', '92020', '92025', '92065', '92083', '92140', 
              '92225', '92226', '92250', '92286', '99354', '99201', '99202', '99203', '99204', '99205',
              '99211', '99212', '99213', '99214', '99215', '99221', '99222', '99223', '99231', '99232', 
              '99233', '99241', '99242', '99243', '99244', '99245', '99251', '99252', '99253', '99254', 
              '99255', '99281', '99282', '99283', '99284', '99285', '99304', '99305', '99306', '99307', 
              '99308', '99309', '99310']

significant_codes = [i[:5] for i in significant.keys()]
print(significant_codes)

['0191T', '65222', '65222', '65855', '65855', '65855', '65855', '66710', '66761', '66761', '66761', '66761', '66761', '66761', '68761', '76514', '76514', '76514', '76514', '76514', '76514', '76514', '92020', '92020', '92020', '92020', '92020', '92020', '92020', '92025', '92025', '92025', '92083', '92083', '92083', '92083', '92083', '92083', '92083', '92100', '92100', '92100', '92100', '92100', '92100', '92100', '92133', '92275', '92275', '92275', '92275', '92275', '92286', '95004', '95004', '95930', '95930', '95930', '95930', '95930', '95930', '99205', '99205', '99214', '99215', '99215', '99215', '68761', '68761', '68761', '68761', '68761', '99215', '65820', '65855', '65855', '67700', '67700', '67961', '76513', '92025', '92025', '92132', '92285', '92286', '95004', '95004', '99215', '99215', '99215']


In [107]:
print("proportion of codes used to generate labels :", len(set(used_codes).intersection(set(signficant_codes)))/len(set(signficant_codes)))

proportion of codes used to generate labels : 0.68


In [110]:
print("New codes not yet used in generating labels")
print(set(signficant_codes) - set(used_codes))

New codes not yet used in generating labels
{'0191T', '66710', '92133', '92132', '92285', '67700', '67961', '65820'}
